In [201]:
#import needed modules
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [159]:
#Livin' in csv city
mjgross = pd.read_csv('D:/Flatiron/Project/Zips/bom.movie_gross.csv/bom.movie_gross.csv')
imdbname = pd.read_csv('D:/Flatiron/Project/Zips/imdb.name.basics.csv/name.basics.csv')
imdbtitle = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.akas.csv/title.akas.csv')
imdbtitle2 = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.basics.csv/title.basics.csv')
imdbcrew = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.crew.csv/title.crew.csv')
imdbprinc = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.principals.csv/title.principals.csv')
imdbrate = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.ratings.csv/title.ratings.csv')
rtinfo = pd.read_csv('D:/Flatiron/Project/Zips/rt.movie_info.tsv/rt.movie_info.tsv', delimiter='\t')
rtreviews = pd.read_csv('D:/Flatiron/Project/Zips/rt.reviews.tsv/rt.reviews.tsv', sep=None)
tmdbmovies = pd.read_csv('D:/Flatiron/Project/Zips/tmdb.movies.csv/tmdb.movies.csv')
tnbudgets = pd.read_csv('D:/Flatiron/Project/Zips/tn.movie_budgets.csv/tn.movie_budgets.csv')

In [160]:
# joined imdbcrew and imdbtitle2
imdbcrewtitle2 = imdbtitle2.join(imdbcrew.set_index('tconst'), on='tconst')


In [161]:
#Editing, dropping mostly empty or useless columns
imdbname.drop(['birth_year','death_year'], axis=1, inplace=True)
imdbtitle.drop(['attributes','language'], axis=1, inplace=True)
imdbprinc.drop(['job','characters'], axis=1, inplace=True)
rtinfo.drop(['synopsis','theater_date','dvd_date','studio','currency'], axis=1, inplace=True)

In [162]:
#Make new dict of genres, cutting out fluff
genre_dict = [{"id":28,"name":"Action"},{"id":12,"name":"Adventure"},{"id":16,"name":"Animation"},{"id":35,"name":"Comedy"},{"id":80,"name":"Crime"},{"id":99,"name":"Documentary"},{"id":18,"name":"Drama"},{"id":10751,"name":"Family"},{"id":14,"name":"Fantasy"},{"id":36,"name":"History"},{"id":27,"name":"Horror"},{"id":10402,"name":"Music"},{"id":9648,"name":"Mystery"},{"id":10749,"name":"Romance"},{"id":878,"name":"Science Fiction"},{"id":10770,"name":"TV Movie"},{"id":53,"name":"Thriller"},{"id":10752,"name":"War"},{"id":37,"name":"Western"}]
k = []
v = []
for i in range(len(genre_dict)):
    k.append(genre_dict[i]['id'])
    v.append(genre_dict[i]['name'])
gendic = dict(zip(k,v))
print(gendic)

{28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}


In [163]:
#Make lists out of Series
titles = tmdbmovies['title'].to_list()
gennums = tmdbmovies['genre_ids'].to_list()

In [164]:
#Create empty DataFrame
genresnp = np.zeros(shape=(26517, 19))
genre = pd.DataFrame(genresnp, columns = v)
genre = genre.applymap(lambda x: int(x))
genre['titles'] = titles
genre = genre.set_index('titles')

In [165]:
#Make list of lists of ints out of list of strings of lists of ints. It was as hard as it is confusing.
gennums = [i.strip("[]").split(", ") for i in gennums] 
gennums = [[int(j) for j in i if j] for i in gennums]

In [166]:
#Use list of int genres as keys to produce names of genres in genre dictionary.
gennums = [[gendic[j] for j in i if j] for i in gennums]
gennums
n= -1
for i in gennums:
    n += 1
    for j in i:
        if j:
            genre.iloc[n].loc[j] += 1
genre

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
titles,,,,,,,,,,,,,,,,,,,
Harry Potter and the Deathly Hallows: Part 1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
How to Train Your Dragon,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
Iron Man 2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Toy Story,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Inception,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Laboratory Conditions,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
_EXHIBIT_84xxx_,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
The Last One,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [167]:
#Just to have a handy reference point
print('\n mjgross \n'); print(len(mjgross)); print(mjgross.isna().sum())
print('\n imdbname \n'); print(len(imdbname)); print(imdbname.isna().sum())
print('\n imdbtitle \n'); print(len(imdbtitle)); print(imdbtitle.isna().sum())
print('\n imdbcrewtitle2 \n'); print(len(imdbcrewtitle2)); print(imdbcrewtitle2.isna().sum())
print('\n imdbprinc \n'); print(len(imdbprinc)); print(imdbprinc.isna().sum())
print('\n imdbrate \n'); print(len(imdbrate)); print(imdbrate.isna().sum())
print('\n rtinfo \n'); print(len(rtinfo)); print(rtinfo.isna().sum())
print('\n rtreviews \n'); print(len(rtreviews)); print(rtreviews.isna().sum())
print('\n tmdbmovies \n'); print(len(tmdbmovies)); print(tmdbmovies.isna().sum())
print('\n tnbudgets \n'); print(len(tnbudgets)); print(tnbudgets.isna().sum())


 mjgross 

3387
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

 imdbname 

606648
nconst                    0
primary_name              0
primary_profession    51340
known_for_titles      30204
dtype: int64

 imdbtitle 

331703
title_id                  0
ordering                  0
title                     0
region                53293
types                163256
is_original_title        25
dtype: int64

 imdbcrewtitle2 

146144
tconst                 0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
directors           5727
writers            35883
dtype: int64

 imdbprinc 

1028186
tconst      0
ordering    0
nconst      0
category    0
dtype: int64

 imdbrate 

73856
tconst           0
averagerating    0
numvotes         0
dtype: int64

 rtinfo 

1560
id               0
rating           3
genre            8
director       1

In [168]:
#Group Box office numbers by studio
studiogross = mjgross.set_index('studio')
mjgross.groupby('studio').domestic_gross.mean().sort_values(ascending=False).head(10)

studio
BV         1.737644e+08
P/DW       1.682900e+08
WB (NL)    8.879333e+07
Uni.       8.777138e+07
WB         8.691461e+07
Fox        8.051103e+07
Sony       7.761177e+07
Par.       7.609773e+07
MGM        6.666667e+07
Sum.       6.212473e+07
Name: domestic_gross, dtype: float64

In [169]:
#list of movies that made over $400 million
top21 = ['Toy story',
 "Marvel's The Avengers",
 'The Dark Knight Rises',
 'The Hunger Games',
 'Frozen',
 'Iron Man 3',
 'The Hunger Games: Catching Fire',
 'Star Wars: The Force Awakens',
 'Jurassic World',
 'Avengers: Age of Ultron',
 'Captain America: Civil War',
 'Rogue One: A Star Wars Story',
 'Finding Dory',
 'Star Wars: The Last Jedi',
 'Beauty and the Beast',
 'Jumanji: Welcome to the Jungle',
 'Wonder Woman',
 'Avengers: Infinity War',
 'Black Panther',
 'Jurassic World: Fallen Kingdom',
 'Incredibles 2']

In [170]:
#list of amounts the movies made
moneybank = [415000000.0,
 623400000,
 448100000,
 408000000,
 400700000,
 409000000,
 424700000,
 936700000,
 652300000,
 459000000,
 408100000,
 532200000,
 486300000,
 620200000,
 504000000,
 404500000,
 412600000,
 678800000,
 700100000,
 417700000,
 608600000]

In [171]:
# New df of genres of top 21 movies
genre21 = genre[~genre.index.duplicated()]
genre21 = genre21.reindex(top21)

In [172]:
# Add columns for domestic gross and budget
genre21 = genre[~genre.index.duplicated()]
genre21 = genre21.reindex(top21)
genre21['domgross'] = moneybank
tnbudgets = tnbudgets.set_index('movie')
tnbudgets21 = tnbudgets[~tnbudgets.index.duplicated()]
tnbudgets21 = tnbudgets21.reindex(top21)
tnbudgets21.shape
budgets21 = list(tnbudgets21['production_budget'])
genre21['budget'] = budgets21

In [173]:
# Clean budget column. Fix NaN later
genre21['budget'] = genre21['budget'].str.replace(',','')
genre21['budget'] = genre21['budget'].fillna(10)
genre21['budget'] = genre21['budget'].apply(lambda x: int(str(x)[1:]))

In [174]:
# Found missing values using original source's website
genre21.loc['Toy story', 'budget'] = 30000000
genre21.loc["Marvel's The Avengers", 'budget'] = 225000000
genre21.loc['Star Wars: The Force Awakens', 'budget'] = 306000000
genre21.loc['Star Wars: The Last Jedi', 'budget'] = 200000000

In [175]:
# Use new columns to create informative columns
genre21['return'] = genre21['domgross'] / genre21['budget'] 
genre21['profit'] = genre21['domgross'] - genre21['budget']

In [206]:
#Researched Rotten Tomatoes website, reformat list to conform to url structure
top = []
for movie in top21:
    movie = movie.replace(' ', '_').replace(':', '').replace("'",'')
    top.append(movie)
# Manually fix two entries that are named differently. Not sure how to automate.
top[14] = 'beauty_and_the_beast_2017'
top[7] = 'star_wars_episode_vii_the_force_awakens'
# Web Scrape RT to find critic and audience ratings for top 21 movies
critic_ratings = []
audience_ratings = []
for i in range(21):
    url = 'http://www.rottentomatoes.com/m/{}'.format(top[i])
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    try:
        critic_ratings.append(soup.find('span', class_="mop-ratings-wrap__percentage").text.strip())
    except:
        critic_ratings.append(top[i])
    try:
        audience_ratings.append(soup.find('span', class_="mop-ratings-wrap__percentage").text.strip())
    except:
        audience_ratings.append(top[i])

In [207]:
# Add two new columns for ratings
genre21['critic_ratings'] = critic_ratings
genre21['audience_ratings'] = audience_ratings

In [208]:
genre21

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,TV Movie,Thriller,War,Western,domgross,budget,return,profit,critic_ratings,audience_ratings
titles,,,,,,,,,,,,,,,,,,,,,
Toy story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,415000000.0,30000000,13.833333,385000000.0,100%,100%
Marvel's The Avengers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,623400000.0,225000000,2.770667,398400000.0,92%,92%
The Dark Knight Rises,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,448100000.0,275000000,1.629455,173100000.0,87%,87%
The Hunger Games,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,408000000.0,80000000,5.100000,328000000.0,84%,84%
Frozen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,400700000.0,150000000,2.671333,250700000.0,56%,56%
Iron Man 3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,409000000.0,200000000,2.045000,209000000.0,79%,79%
The Hunger Games: Catching Fire,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,424700000.0,130000000,3.266923,294700000.0,90%,90%
Star Wars: The Force Awakens,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,936700000.0,306000000,3.061111,630700000.0,92%,92%
Jurassic World,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,652300000.0,215000000,3.033953,437300000.0,71%,71%


In [198]:
tmdbmovies

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [196]:
imdbcrewtitle2['primary_title'][0]

'Sunghursh'

In [ ]:
imdbprincname = imdbprinc.merge(imdbname, on='nconst')

In [ ]:
imdbprincnametitleouter = imdbprincname.merge(imdbtitle, left_on='tconst', right_on='title_id',how='outer')

In [178]:
imdball = imdbprincnametitleouter.merge(imdbcrewtitle2, on='tconst')
imdball

NameError: name 'imdbprincnametitleouter' is not defined